In [144]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
#import matplotlib.pyplot as plt
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
import pickle
from sklearn.model_selection import train_test_split
import torch.optim as optim


In [145]:
#load the shadow model trained in the other python script
device = "cpu"
resnet_shadow =  models.mobilenet_v2(pretrained=False)
num_classes = 10  # Change this to the number of classes in your dataset
num_features = resnet_shadow.classifier[1].in_features
resnet_shadow.classifier = nn.Sequential(
    nn.Linear(num_features, num_classes)
)

resnet_cifar = torch.load("mobilenet_shadow_cifar_overtrained.pth",map_location=torch.device('cpu'))

resnet_shadow.load_state_dict(resnet_cifar)

<All keys matched successfully>

In [146]:
DATA_PATH = 'shadow.p'
# Change the DATA_PATH to your local pickle file path

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

with open(DATA_PATH, "rb") as f:
    dataset = pickle.load(f)


#splitting
#only use train set here
train_data, val_data = train_test_split(dataset, test_size=(1-0.5),shuffle=False)
  
dataloader = torch.utils.data.DataLoader(
    train_data, batch_size=
     1 , shuffle=False, num_workers=2)
testloader =  torch.utils.data.DataLoader(val_data, batch_size=1,
                                          shuffle=True, num_workers=2)

for batch_idx, (img, label) in enumerate(dataloader):
    img = img.to(device)


In [147]:
#generate dataset for attack model
resnet_shadow.eval()
dataset_attack = []
with torch.no_grad():
    for images, labels in testloader: #need only one
            # Move images and labels to the appropriate device
        images, labels = images.to(device), labels.to(device)
            
            # Forward pass
        logits = resnet_shadow(images)
        
        #take the 3 biggest logist
        
        top_values = torch.topk(logits, k=3).values
        top_values, indices = torch.sort(top_values, dim=1, descending=True)
        dataset_attack.append([top_values,0])
        
with torch.no_grad():
    for images, labels in dataloader: #need only one
            # Move images and labels to the appropriate device
        images, labels = images.to(device), labels.to(device)
            
            # Forward pass
        logits = resnet_shadow(images)
        
        #take the 3 biggest logist
        
        top_values = torch.topk(logits, k=3).values
        top_values, indices = torch.sort(top_values, dim=1, descending=True)

        dataset_attack.append([top_values,1])
        



In [148]:
standardized_data_list = dataset_attack
"""
# Convert all tensors to the same dtype first
tensors = [data[0].float() for data in dataset_attack]  # Ensure all tensors are Float type
all_data = torch.cat(tensors, dim=0)  # Concatenate all tensors

# Calculate mean and std
mean = all_data.mean(dim=0)
std = all_data.std(dim=0)

# Standardize data in the list
standardized_data_list = [( (data[0] - mean) / std, data[1] ) for data in dataset_attack]
"""

'\n# Convert all tensors to the same dtype first\ntensors = [data[0].float() for data in dataset_attack]  # Ensure all tensors are Float type\nall_data = torch.cat(tensors, dim=0)  # Concatenate all tensors\n\n# Calculate mean and std\nmean = all_data.mean(dim=0)\nstd = all_data.std(dim=0)\n\n# Standardize data in the list\nstandardized_data_list = [( (data[0] - mean) / std, data[1] ) for data in dataset_attack]\n'

In [149]:
dataloader_attack = torch.utils.data.DataLoader(
    
    standardized_data_list, batch_size=64, shuffle=True, num_workers=2) #shuffled training data

In [150]:
import torch.optim as optim

class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(3, 32)
        self.bn1 = nn.BatchNorm1d(32)  # Matches the output of fc1
        self.fc2 = nn.Linear(32, 64)
        self.bn2 = nn.BatchNorm1d(64)  # Matches the output of fc2
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)  # Matches the output of fc3
        self.fc4 = nn.Linear(32, 16)
        self.bn4 = nn.BatchNorm1d(16)  # Matches the output of fc4
        self.fc5 = nn.Linear(16, 8)
        self.bn5 = nn.BatchNorm1d(8)   # Matches the output of fc5
        self.fc6 = nn.Linear(8, 1)
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = torch.relu(self.bn3(self.fc3(x)))
        x = torch.relu(self.bn4(self.fc4(x)))
        x = self.dropout(torch.relu(self.bn5(self.fc5(x))))
        x = self.sigmoid(self.fc6(x))
        return x

# Ensure input dimension is correct


In [151]:
model = SimpleNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [152]:
optimizer = optim.Adam(model.parameters(), lr=0.0001)


In [153]:
# Define your training function

def train(model, dataloader, optimizer, criterion, epochs):
    for inputs, labels in dataloader:
        inputs = inputs.float()  # Ensures input tensors are floats
        labels = labels.float().view(-1, 1)  # Ensures labels are floats and reshaped correctly

        optimizer.zero_grad()
        outputs = model(inputs.squeeze(dim = 1))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item()}")

In [154]:
model.train()

SimpleNN(
  (fc1): Linear(in_features=3, out_features=32, bias=True)
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=32, out_features=64, bias=True)
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (bn3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=32, out_features=16, bias=True)
  (bn4): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc5): Linear(in_features=16, out_features=8, bias=True)
  (bn5): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc6): Linear(in_features=8, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (sigmoid): Sigmoid()
)

In [155]:
#dict =  torch.load('attack_model.pth_3', map_location='cpu')
#model.load_state_dict(dict)

In [156]:
"""for inputs in dataloader_attack:
    inputs = inputs[0]
    print(inputs.shape)  # Check input shape consistenc
    print(inputs)
    print(inputs)
    outputs = model(inputs.squeeze(dim = 1))"""

'for inputs in dataloader_attack:\n    inputs = inputs[0]\n    print(inputs.shape)  # Check input shape consistenc\n    print(inputs)\n    print(inputs)\n    outputs = model(inputs.squeeze(dim = 1))'

In [185]:
model.train()
train(model,dataloader_attack,optimizer,criterion,4000)

Loss: 0.6902701258659363
Loss: 0.6743139624595642
Loss: 0.6922743916511536
Loss: 0.6667091846466064
Loss: 0.6869983673095703
Loss: 0.7223072648048401
Loss: 0.6834421753883362
Loss: 0.6674128770828247
Loss: 0.6828210353851318
Loss: 0.7042727470397949
Loss: 0.6898802518844604
Loss: 0.7122758030891418
Loss: 0.7144584059715271
Loss: 0.6870189905166626
Loss: 0.6901620626449585
Loss: 0.6994820833206177
Loss: 0.701591968536377
Loss: 0.6752540469169617
Loss: 0.680755078792572
Loss: 0.666269838809967
Loss: 0.7113248109817505
Loss: 0.6863154172897339
Loss: 0.6828051805496216
Loss: 0.7177075743675232
Loss: 0.6980864405632019
Loss: 0.6898539662361145
Loss: 0.7343880534172058
Loss: 0.6785268783569336
Loss: 0.6828323602676392
Loss: 0.6908277869224548
Loss: 0.7017245292663574
Loss: 0.6839150190353394
Loss: 0.6825065612792969
Loss: 0.713539719581604
Loss: 0.681577205657959
Loss: 0.6963977217674255
Loss: 0.7059194445610046
Loss: 0.6956728100776672
Loss: 0.6964032053947449
Loss: 0.7000255584716797
Loss:

In [158]:
torch.save(model.state_dict(), 'attack_model_mobilenet_cifar.pth')

<h1>Evaluation</h1>

In [159]:
DATA_PATH = "eval.p"

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

with open(DATA_PATH, "rb") as f:
    dataset = pickle.load(f)
# Convert all tensors to the same dtype first

dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=1 , shuffle=False, num_workers=2)
#splitting
for batch_idx, (img, label, membership) in enumerate(dataloader):
    img = img.to(device)


# Define your evaluation function


In [160]:
dataset = []
resnet_shadow.eval()
with torch.no_grad():
    for images,_, b in dataloader: #need only one
            # Move images and labels to the appropriate device
        images, labels = images.to(device), labels.to(device)
            
            # Forward pass
        logits = resnet_shadow(images)
        print(logits)
        
        #take the 3 biggest logist
        
        top_values = torch.topk(logits, k=3).values
        

        dataset.append([top_values,b])

tensor([[-2.7103, -4.6378, -0.1289,  2.9930,  1.0985,  5.1932, -1.2353, -0.8289,
         -3.7446, -4.0443]])
tensor([[ 2.4306, -2.3788, -0.4651, -3.2092,  1.2412, -1.4609, -6.4277,  1.3404,
          0.1524, -0.1557]])
tensor([[-5.4066, -2.9340,  1.5373, -0.4616, -2.1943,  2.6823,  2.5824,  2.1791,
         -3.8598, -2.3940]])
tensor([[-1.7397,  5.6713, -4.5093, -0.4556, -1.6378, -5.0307, -0.3070,  4.7039,
         -5.1625, -1.9458]])
tensor([[ 0.7037, -1.1619, -0.0768,  1.4369, -0.4303, -2.4882, -2.0719, -0.5609,
          2.1269, -2.9825]])
tensor([[-0.7748, -1.5942, -1.1614, -0.5312, -3.9587, -2.6068,  5.3597,  2.3739,
         -4.1059, -0.3337]])
tensor([[ 2.3683, -1.8953, -3.5252, -4.8749,  3.6337, -1.5636, -0.2816, -2.9056,
          3.8003, -3.8218]])
tensor([[-3.7642, -0.1180, -4.1284, -0.2648,  0.5093,  0.9776,  2.6419,  1.4606,
         -4.5504, -0.5863]])
tensor([[-0.3845, -2.8281, -3.8035, -3.1389,  3.0722, -3.4887, -5.4458,  6.7233,
         -2.1374,  2.2141]])
tensor([[ 

In [161]:
resnet_target_model =  models.mobilenet_v2(pretrained=False)
num_classes = 10  # Change this to the number of classes in your dataset
num_features = resnet_target_model.classifier[1].in_features
resnet_target_model.classifier = nn.Sequential(
    #nn.Dropout(p=0.),  # Optional dropout layer for regularization
    nn.Linear(num_features, num_classes)
)
check = torch.load("mobilenetv2_cifar10.pth", map_location=torch.device('cpu'))
check["net"]["classifier.0.weight"] =check["net"].pop("classifier.1.weight")
check["net"]["classifier.0.bias"] =check["net"].pop("classifier.1.bias")

resnet_target_model.load_state_dict(check["net"])
resnet_target_model.eval()

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [162]:
#get posteriors from target
dataset_eval = []
with torch.no_grad():
    for images,_, member in dataloader: #need only one
            # Move images and labels to the appropriate device
        images, labels = images.to(device), labels.to(device)
            
            # Forward pass
        logits = resnet_target_model(images)
        
        #take the 3 biggest logist
        
        top_values = torch.topk(logits, k=3).values #order poseri
        sorted_tensor, indices = torch.sort(top_values, dim=1,descending=True)
        dataset_eval.append([sorted_tensor, member.item()])
        


In [163]:
dataset = dataset_eval
"""# Convert all tensors to the same dtype first
tensors = [data[0].float() for data in dataset_eval]  # Ensure all tensors are Float type
all_data = torch.cat(tensors, dim=0)  # Concatenate all tensors

# Calculate mean and std
mean = all_data.mean(dim=0)
std = all_data.std(dim=0)

# Standardize data in the list
dataset = [( (data[0] - mean) / std, data[1] ) for data in dataset_eval]"""

'# Convert all tensors to the same dtype first\ntensors = [data[0].float() for data in dataset_eval]  # Ensure all tensors are Float type\nall_data = torch.cat(tensors, dim=0)  # Concatenate all tensors\n\n# Calculate mean and std\nmean = all_data.mean(dim=0)\nstd = all_data.std(dim=0)\n\n# Standardize data in the list\ndataset = [( (data[0] - mean) / std, data[1] ) for data in dataset_eval]'

In [164]:
dataloader_eval = torch.utils.data.DataLoader(
    
    dataset, batch_size=1 , shuffle=False, num_workers=2)

In [186]:
def evaluate(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation
        for data in dataloader:
            inputs = data[0]
            
            labels = data[1].float()
            labels = torch.tensor(labels).float()
            #labels = torch.tensor(labels[0])
            optimizer.zero_grad()

            outputs = model(inputs.squeeze(dim=1))
            predicted = torch.round(outputs)  # Round the outputs to 0 or 1
            #predicted = abs(predicted -1)
            print(predicted,labels)

            total += labels.size(0)  # Increment the total count by batch size
            correct += (predicted == labels).sum().item()  # Count correct predictions

    accuracy = correct / total
    return accuracy

# Assuming you have a DataLoader called 'test_loader' for evaluation
# Create an instance of your model

# Assuming 'test_loader' is your DataLoader for evaluation
accuracy = evaluate(model, dataloader_eval)#


print(f'Accuracy: {accuracy:.2f}')

/tmp/ipykernel_7559/3541459405.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).float()


tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tenso